In [1]:
import numpy as np 
import math

In [3]:
# L - Length of the input sequence 
# d_k,d_v - Size of each of the vector for illustrative purpose it's 8
L,d_k,d_v = 4,8,8
q = np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_v)

In [6]:
# Using the randn function - values are assigned from the normal distribution 
print("Q\n",q) # 8 x 1 - for Every single word - Totally 8 x 4
print("K\n",k) # 8 x 1 - for Every single word - Totally 8 x 4
print("V\n",v) # 8 x 1 - for Every single word - Totally 8 x 4

Q
 [[ 0.44008364 -0.65893923 -0.82743603  2.07134499 -0.756638    0.18606377
  -2.52939241 -0.99671064]
 [ 0.45847306  0.13226699  0.39716341 -0.35557593  0.92675916  0.86731148
  -1.0848844  -0.75466212]
 [-1.73679108  0.50542825  0.67550296 -0.48343931 -0.78572117 -0.57689201
  -0.52532605 -1.41217445]
 [-2.09387521 -0.89363413 -0.59569919 -1.71644659 -0.36390696 -0.30201351
   0.22337068 -0.50049467]]
K
 [[-0.97577445  0.8762965  -2.38211946 -1.7001989  -2.1416398   0.14183991
   0.82795221  0.06947477]
 [ 0.75104729  0.51849052  0.78832873  0.55552962  1.34575561  0.47145897
   0.51505236  1.26781612]
 [-0.43040316  1.29402735 -0.45535669 -0.2645018   0.40588437 -0.22652967
   0.51160386 -1.89667442]
 [ 0.13683429 -1.27318423 -0.18401422  1.90297292 -0.61696554  1.93138142
  -0.68380964  0.23601658]]
V
 [[ 1.5062644  -1.35215475  0.48031349 -0.28786724 -0.40148487  2.23991922
  -2.90143267 -1.0343477 ]
 [ 1.38140753 -0.31255013  1.31883032 -0.84232214 -2.56852263  0.81663448
   1.7

# For the Formula of Self Attention - Refer to the Notes

In [8]:
# Self Attention - Initially from the encoder side we need all the words to look at other words so that we can check the similarity/affinity with each other
# Coming to the First Step 
np.matmul(q,k.T)
# for each row we have to look at the max value - that's the one each row will focus on 
# my - this will focus on the 3rd index
# name - this will focus on the 2nd index
# is - this will focus on the 2nd index 
# Vibhav - this will focus on the 1st index

array([[-3.07412046, -3.00967196, -0.96606258,  7.31370534],
       [-3.48542943,  0.66902454,  0.94303088,  0.81167197],
       [ 2.41828564, -4.16871647,  3.44328164, -2.5289391 ],
       [ 6.48407724, -4.61068825,  1.45434592, -2.93514059]])

In [11]:
# Why do we need the denominator - SquareRoot(d_k)
# Answer - In order to minimize the variance and hence stablize the values of the multiplication(q,k.T) vector
q.var(), k.var(), np.matmul(q,k.T).var()
# As you can see below the difference of the first two values is comparitively less 
# Whereas the last value is far way distant from the first two values

(0.8851675882015266, 1.1668235844412789, 12.65558917253314)